<a href="https://colab.research.google.com/github/hansong0219/Advanced-DeepLearning-Study/blob/master/UNET/UNet_upsample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import sys
from tensorflow.keras.layers import Input, Dropout, Concatenate
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, UpSampling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LeakyReLU, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
import matplotlib.pyplot as plt
import tensorflow as tf

import cv2

In [ ]:
import tensorflow as tf 
physical_devices =tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)
tf.config.experimental.set_memory_growth(physical_devices[1],True)

In [ ]:
def down_sample(layer_inputs,filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0.,0.02)
    d = Conv2D(filters, size, strides=1,padding='same', kernel_initializer=initializer, use_bias=False)(layer_inputs)
    if apply_batchnorm:
        d = BatchNormalization()(d)
    
    d = LeakyReLU(alpha=0.2)(d)
    d = MaxPooling2D(pool_size=(2, 2))(d)
    return d

def up_sample(layer_inputs, skip_input,filters, size, dropout_rate=0):
    initializer = tf.random_normal_initializer(0.,0.02)
    u = UpSampling2D(size=(2,2), interpolation = 'bilinear')(layer_inputs)
    u = Conv2D(filters, size, strides=1,padding='same', kernel_initializer=initializer,use_bias=False)(u)
    if dropout_rate:
        u = Dropout(dropout_rate)(u)
    
    u = tf.keras.layers.ReLU()(u)
    u = Concatenate()([u, skip_input])
    return u

In [ ]:
def Build_UNET():
    input_shape = (256,256,3)
    output_channel = 3
    inputs = Input(shape=input_shape,name="inputs")
    
    d1 = down_sample(inputs, 64, 3, apply_batchnorm=False) #(128,128,3)
    d2 = down_sample(d1, 128, 3) #(64,64,128)
    d3 = down_sample(d2, 256, 3)
    d4 = down_sample(d3, 512, 3)
    d5 = down_sample(d4, 512, 3)
    d6 = down_sample(d5, 512, 3)
    d7 = down_sample(d6, 512, 3)
    d8 = down_sample(d7, 512, 3)
    
    u7 = up_sample(d8, d7, 512, 3, dropout_rate = 0.5)
    u6 = up_sample(u7, d6, 512, 3, dropout_rate = 0.5)
    u5 = up_sample(u6, d5, 512, 3)
    u4 = up_sample(u5, d4, 512, 3)
    u3 = up_sample(u4, d3, 256, 3)
    u2 = up_sample(u3, d2, 128, 3)
    u1 = up_sample(u2, d1, 64, 3)
    
    initializer = tf.random_normal_initializer(0.,0.02)
    u0 = UpSampling2D(size=(2,2),interpolation='bilinear')(u1)
    outputs = Conv2D(output_channel,
                     kernel_size=4, 
                     strides=1, 
                     padding='same', 
                     kernel_initializer=initializer, 
                     activation='tanh')(u0)
    
    return Model(inputs, outputs)

In [ ]:
unet = Build_UNET()

In [ ]:
loss=BinaryCrossentropy(from_logits=True)
optimizer = Adam(1e-4, beta_1=0.5)
unet.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])